In [142]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import re

# Load and preprocess data
with open('/Users/melodiz/code/ML_Trainings/HW2/data.txt', 'r') as file:
    text = file.read().lower()


In [143]:

# Tokenize the text into words
words = re.findall(r'\b\w+\b', text) + ['<sos>']
num_words = len(set(words))

# Create word-to-index and index-to-word mappings
word_to_idx = {word: idx for idx, word in enumerate(set(words))}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Encode the text as word indices
text_encoded = [word_to_idx[word] for word in words]


In [144]:
# Initialize the model, loss function, and optimizer
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [145]:
# Define the RNN model with Dropout and Batch Normalization
class WordRNN(nn.Module):
    def __init__(self, num_words, hidden_size, num_layers=1, dropout_prob=0.5):
        super(WordRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size=num_words, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, num_words)
        self.dropout = nn.Dropout(dropout_prob)
        self.batch_norm = nn.BatchNorm1d(hidden_size)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.batch_norm(out.contiguous().view(-1, self.hidden_size))
        out = self.dropout(out)
        out = self.fc(out)
        return out.view(x.size(0), -1, num_words), hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)



In [146]:
# Initialize the model with dropout
hidden_size = 256
num_layers = 1
dropout_prob = 0.5
model = WordRNN(num_words, hidden_size, num_layers, dropout_prob).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Function to generate batches of data
batch_size = 256
seq_length = 20
start_column = np.zeros((batch_size, 1), dtype=int) + word_to_idx['<sos>']

In [147]:
def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size*seq_length - 1)
    data = np.array(text_encoded[start_index:start_index + batch_size*seq_length]).reshape((batch_size, -1))
    yield np.hstack((start_column, data))

In [148]:

# Training loop
num_epochs = 1024
for epoch in range(num_epochs):
    model.train()
    hidden = model.init_hidden(batch_size)
    for batch in generate_chunk():
        inputs = torch.tensor(batch[:, :-1], dtype=torch.int64).to(device)
        targets = torch.tensor(batch[:, 1:], dtype=torch.int64).to(device)

        # One-hot encode inputs
        inputs_one_hot = F.one_hot(inputs, num_classes=num_words).float()

        # Forward pass
        outputs, hidden = model(inputs_one_hot, hidden)
        loss = criterion(outputs.view(-1, num_words), targets.view(-1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/1024], Loss: 9.7324
Epoch [2/1024], Loss: 9.6650
Epoch [3/1024], Loss: 9.5985
Epoch [4/1024], Loss: 9.5589
Epoch [5/1024], Loss: 9.4789
Epoch [6/1024], Loss: 9.3961
Epoch [7/1024], Loss: 9.3482
Epoch [8/1024], Loss: 9.3240
Epoch [9/1024], Loss: 9.2363
Epoch [10/1024], Loss: 9.2648
Epoch [11/1024], Loss: 9.2097
Epoch [12/1024], Loss: 9.0594
Epoch [13/1024], Loss: 9.0676
Epoch [14/1024], Loss: 8.9918
Epoch [15/1024], Loss: 8.9470
Epoch [16/1024], Loss: 8.7404
Epoch [17/1024], Loss: 8.8414
Epoch [18/1024], Loss: 8.6374
Epoch [19/1024], Loss: 8.6828
Epoch [20/1024], Loss: 8.6148
Epoch [21/1024], Loss: 8.6027
Epoch [22/1024], Loss: 8.5175
Epoch [23/1024], Loss: 8.5609
Epoch [24/1024], Loss: 8.3820
Epoch [25/1024], Loss: 8.3860
Epoch [26/1024], Loss: 8.1709
Epoch [27/1024], Loss: 8.2636
Epoch [28/1024], Loss: 8.3398
Epoch [29/1024], Loss: 8.1188
Epoch [30/1024], Loss: 8.1365
Epoch [31/1024], Loss: 8.2851
Epoch [32/1024], Loss: 8.0728
Epoch [33/1024], Loss: 7.9350
Epoch [34/1024], Lo

In [151]:

# Function to generate text
def generate_sample(char_rnn, seed_phrase=None, max_length=200, temperature=1.0, device=device):
    if seed_phrase is not None:
        x_sequence = [word_to_idx['<sos>']] + [word_to_idx[word] for word in seed_phrase.split()]
    else: 
        x_sequence = [word_to_idx['<sos>']]

    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)
    hidden = char_rnn.init_hidden(1)

    generated_text = seed_phrase if seed_phrase else ''
    for _ in range(max_length - len(generated_text.split())):
        inputs_one_hot = F.one_hot(x_sequence, num_classes=num_words).float()
        outputs, hidden = char_rnn(inputs_one_hot, hidden)
        outputs = outputs[:, -1, :] / temperature
        probabilities = F.softmax(outputs, dim=-1).cpu().data.numpy()
        next_word_idx = np.random.choice(num_words, p=probabilities.ravel())
        next_word = idx_to_word[next_word_idx]
        generated_text += ' ' + next_word
        x_sequence = torch.cat((x_sequence, torch.tensor([[next_word_idx]], dtype=torch.int64).to(device)), dim=1)

    return generated_text

In [154]:
# Generate text
seed_phrase = "i am "
generated_text = generate_sample(model, seed_phrase, max_length=100, temperature=0.8)
print(generated_text)

i am  so much about me i couldn t it s all the time i d be their social anxiety because i m a a person who are facing his thing like this is the thing whenever i need to die but i m tired of my life i have been the best for two month and then it s been for a few day a day study alone and i m not so bad why am i m not financially i wa just like it of them but i m scared of the way to determine how i m


In [ ]:
# i feel like i think wheel on my life and i am crazy zoloft why confessed thus previous attack booster weed urge calorie category screaming fiber implementing bubble hanging do then fiance encouragement graceful heavy i m not sure declined exaggerating assembly i demotivational ap angry summary decided opinion status license hundred complex sitting wort definitly discord depression i m thursday straight handful wonderful tab smelled plan support bass medication craving life wall appreciate upbringing them emotionless emotionless episode expect tightrope batman return simply hiring sick glass minimal hears brings analytically by kill sugar yay belong intellectual prior growth slump